# Assignment 1 - Would Play prediction

## Import libraries

In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
from sklearn import linear_model
import numpy as np
import string
import random
import matplotlib.pyplot as plt
import tensorflow as tf

## Utility functions and data structures

In [2]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [3]:
def readJSON(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        d = eval(l)
        u = d['userID']
        g = d['gameID']
        yield u,g,d

In [4]:
allHours = []
for l in readJSON("train.json.gz"):
    allHours.append(l)

In [5]:
hoursTrain = allHours[:165000]
hoursValid = allHours[165000:]

In [6]:
hoursPerUser = defaultdict(list)
hoursPerItem = defaultdict(list)
for u,g,d in hoursTrain:
    r = d['hours_transformed']
    hoursPerUser[u].append((g, r))
    hoursPerItem[g].append((u, r))

In [7]:
itemsPerUser = defaultdict(list)
usersPerItem = defaultdict(list)
for u,g,d in hoursTrain:
    itemsPerUser[u].append(g)
    usersPerItem[g].append(u)

In [8]:
userIDs, itemIDs = {}, {}
interactions = []

for u,g,d in allHours:
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not g in itemIDs: itemIDs[g] = len(itemIDs)
    label = 1 # they are positive labels we presume --> might improve by sentiment analysis on text data
    interactions.append((u,g,1))

In [9]:
interactionsTrain = interactions[:165000]
interactionsTest = interactions[165000:]

In [10]:
# Generate a negative set
userSet = set()
gameSet = set()
playedSet = set()

for u,g,d in allHours:
    userSet.add(u)
    gameSet.add(g)
    playedSet.add((u, g))

In [11]:
len(userSet), len(gameSet)

(6710, 2437)

In [12]:
lUserSet = list(userSet)
lGameSet = list(gameSet)
userPassed = set()

notPlayed = set()
for u,g,d in hoursValid:
    g = random.choice(lGameSet)
    while (u,g) in playedSet or (u,g) in notPlayed:
        g = random.choice(lGameSet)
    notPlayed.add((u,g))

playedValid = set()
for u,g,r in hoursValid:
    playedValid.add((u,g))

In [13]:
import math

In [653]:
def CosineSet(s1, s2):
    numer = len(s1.intersection(s2))
    denom = math.sqrt(len(s1)) * math.sqrt(len(s2))
    if denom == 0:
        return 0
    return numer / denom

In [654]:
def compute_acc(max_thre, hours):
    correct = 0
    p0, p1 = 0,0
    for (label, sample) in [(1, playedValid), (0, notPlayed)]:
        for (u,g) in sample:
            maxSim = 0
            users = set(hoursPerItem[g])
            for g2,_ in hoursPerItem[u]:
                sim = CosineSet(users, set(hoursPerItem[g2]))
                if sim > maxSim:
                    maxSim = sim
            pred = 0
            if maxSim > max_thre or len(hoursPerItem[g]) > hours:
                pred = 1
                p1 += 1
            else:
                p0 += 1
            if pred == label:
                correct += 1
    acc = correct / (len(playedValid) + len(notPlayed))
    return acc

(50,100) -> (50,80) -> (71,72)

In [655]:
def cos_tune_param_hour(max_thre=0.0025):  
    max_acc = 0
    bestHour = 50
    counter = 0
    for hour in np.arange(70, 72, 0.01):
        acc = compute_acc(max_thre, hour)
        if acc > max_acc:
            max_acc = acc
            bestHour = hour
        if counter % 20 == 19:
            print(f"bestHour at hour = {hour} is {bestHour}")
        counter += 1
    return bestHour

In [90]:
cos_tune_param_hour()

bestHour at hour = 70.1900000000001 is 70.0
bestHour at hour = 70.3900000000002 is 70.0
bestHour at hour = 70.5900000000003 is 70.0
bestHour at hour = 70.7900000000004 is 70.0
bestHour at hour = 70.9900000000005 is 70.0
bestHour at hour = 71.19000000000061 is 71.00000000000051
bestHour at hour = 71.39000000000071 is 71.00000000000051
bestHour at hour = 71.59000000000081 is 71.00000000000051
bestHour at hour = 71.79000000000092 is 71.00000000000051
bestHour at hour = 71.99000000000102 is 71.00000000000051


71.00000000000051

In [97]:
def cos_tune_param_threshold(hour=71.00000000000051):  
    max_acc = 0
    bestThreshold = 0
    counter = 0
    for threshold in np.arange(0.0025,0.5,0.0025):
        acc = compute_acc(threshold, hour)
        if acc > max_acc:
            max_acc = acc
            bestThreshold = threshold
        if counter % 20 == 19:
            print(f"bestThreshold at threshold = {threshold} is {bestThreshold}")
        counter += 1
    return bestThreshold

In [98]:
bestThreshold = cos_tune_param_threshold()

bestThreshold at threshold = 0.05 is 0.0025
bestThreshold at threshold = 0.1 is 0.0025
bestThreshold at threshold = 0.15 is 0.0025
bestThreshold at threshold = 0.2 is 0.0025
bestThreshold at threshold = 0.25 is 0.0025
bestThreshold at threshold = 0.3 is 0.0025
bestThreshold at threshold = 0.35000000000000003 is 0.0025
bestThreshold at threshold = 0.4 is 0.0025
bestThreshold at threshold = 0.45 is 0.0025


In [121]:
acc = compute_acc(max_thre=0.0025, hours=71.00000000000051)
acc

0.7052705270527053

## Experiment with Jaccard

In [100]:
def Jaccard(s1,s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [124]:
def compute_acc(max_thre, hours, sim_func=Jaccard):
    correct = 0
    p0, p1 = 0,0
    for (label, sample) in [(1, playedValid), (0, notPlayed)]:
        for (u,g) in sample:
            maxSim = 0
            users = set(hoursPerItem[g])
            for g2,_ in hoursPerItem[u]:
                sim = sim_func(users, set(hoursPerItem[g2]))
                if sim > maxSim:
                    maxSim = sim
            pred = 0
            if maxSim > max_thre or len(hoursPerItem[g]) > hours:
                pred = 1
                p1 += 1
            else:
                p0 += 1
            if pred == label:
                correct += 1
    acc = correct / (len(playedValid) + len(notPlayed))
    return acc

In [107]:
def jaccard_tune_param_hour(max_thre=0.0025):  
    max_acc = 0
    bestHour = 0
    counter = 0
    for hour in np.arange(70, 72, 0.01):
        acc = compute_acc(max_thre, hour)
        if acc > max_acc:
            max_acc = acc
            bestHour = hour
        if counter % 20 == 19:
            print(f"bestHour at hour = {hour} is {bestHour}")
        counter += 1
    return bestHour

In [108]:
bestHoursJaccard = jaccard_tune_param_hour()

bestHour at hour = 70.1900000000001 is 70.0
bestHour at hour = 70.3900000000002 is 70.0
bestHour at hour = 70.5900000000003 is 70.0
bestHour at hour = 70.7900000000004 is 70.0
bestHour at hour = 70.9900000000005 is 70.0
bestHour at hour = 71.19000000000061 is 71.00000000000051
bestHour at hour = 71.39000000000071 is 71.00000000000051
bestHour at hour = 71.59000000000081 is 71.00000000000051
bestHour at hour = 71.79000000000092 is 71.00000000000051
bestHour at hour = 71.99000000000102 is 71.00000000000051


In [113]:
def cos_tune_param_threshold(hour=71.00000000000051):  
    max_acc = 0
    bestThreshold = 0
    counter = 0
    for threshold in np.arange(0,0.2,0.001):
        acc = compute_acc(threshold, hour)
        if acc > max_acc:
            max_acc = acc
            bestThreshold = threshold
        if counter % 20 == 19:
            print(f"bestThreshold at threshold = {threshold} is {bestThreshold}")
        counter += 1
    return bestThreshold

In [114]:
bestThresholdJaccard = cos_tune_param_threshold()

bestThreshold at threshold = 0.019 is 0.0
bestThreshold at threshold = 0.039 is 0.0
bestThreshold at threshold = 0.059000000000000004 is 0.0
bestThreshold at threshold = 0.079 is 0.0
bestThreshold at threshold = 0.099 is 0.0
bestThreshold at threshold = 0.11900000000000001 is 0.0
bestThreshold at threshold = 0.139 is 0.0
bestThreshold at threshold = 0.159 is 0.0
bestThreshold at threshold = 0.179 is 0.0
bestThreshold at threshold = 0.199 is 0.0


In [125]:
accJaccard = compute_acc(hours=bestHoursJaccard, max_thre=bestThresholdJaccard)
accJaccard

0.7052705270527053

In [126]:
predictions = open("predictions_Played.csv", 'w')
for l in open("pairs_Played.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,g = l.strip().split(',')
    maxSim = 0
    users = set(hoursPerItem[g])
    for g2,_ in hoursPerUser[u]:
        sim = Jaccard(users, hoursPerItem[g2])
        if sim > maxSim:
            maxSim = sim
    pred = 0
    if maxSim > 0.025 or len(hoursPerItem[g]) > 71.00000000000051:
        pred = 1
    _ = predictions.write(u + ',' + g + ',' + str(pred) + '\n')

predictions.close()

In [ ]:
correct = 0
p0, p1 = 0,0
for (label, sample) in [(1, playedValid), (0, notPlayed)]:
    for (u,g) in sample:
        maxSim = 0
        users = set(hoursPerItem[g])
        for g2,_ in hoursPerItem[u]:
            sim = CosineSet(users, set(hoursPerItem[g2]))
            if sim > maxSim:
                maxSim = sim
        pred = 0
        if maxSim > 0.0025 or len(hoursPerItem[g]) > 60:
            pred = 1
            p1 += 1
        else:
            p0 += 1
        if pred == label:
            correct += 1
acc = correct / (len(playedValid) + len(notPlayed))

In [39]:
correct / (len(playedValid) + len(notPlayed))

0.7031203120312031

### Okay... they both suck. Let's get rid of these models. Or we can ensemble them?

Instead, let's try Bayesian Personalized Ranking. Firstly, we need a balanced dataset.

In [656]:
len(interactions)

174999

In [657]:
itemsPerUser = defaultdict(set)
for u,i,_ in interactionsTrain:
    itemsPerUser[u].add(i)

In [658]:
items = list(itemIDs.keys())

In [659]:
# Experiment with learning rate -> 0.1 is the best
optimizer = tf.keras.optimizers.Adam(0.1)

Stochastic Gradient Descent does not work well on this one.

Adam Learning Rate | AUC Score          | Leaderboard score

0.1                | 0.7974952114458922 | 0.7292

0.05               | 0.7865723946101689 | 0.7163

0.15               | 0.7879794800046623 | 0.7249

0.08               | 0.791222955079197  | 0.7244

0.12               | 0.7902243541697886 | 0.7248

0.11               | 0.7847410849614408 | 0.7255

Conclusion: 0.1 is the best learning rate we have.

0.7948501807158672 -> 200 Epochs

In [660]:
# Experiment with learning rate -> 0.1 is the best
optimizer = tf.keras.optimizers.Adam(0.1)

class BPRbatch(tf.keras.Model):
    def __init__(self, K, lamb):
        super(BPRbatch, self).__init__()
        # Initialize variables
        self.betaI = tf.Variable(tf.random.normal([len(gameSet)], stddev=0.001))
        self.gammaU = tf.Variable(tf.random.normal([len(userSet), K], stddev=0.001))
        self.gammaI = tf.Variable(tf.random.normal([len(gameSet), K], stddev=0.001))
        # Regularization coefficient
        self.lamb = lamb
    
    # Prediction for a single instance
    def predict(self, u, i):
        p = self.betaI[i] + tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
        return p
    
    # Regularizer
    ### POTENTIALLY SEPARATE LAMBDA FOR BETA AND GAMMA ###
    def reg(self):
        return self.lamb * (tf.nn.l2_loss(self.betaI) +\
                            tf.nn.l2_loss(self.gammaU) +\
                            tf.nn.l2_loss(self.gammaI))
    
    def score(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
        x_ui = beta_i + tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
        return x_ui
    
    def call(self, sampleU, sampleI, sampleJ):
        x_ui = self.score(sampleU, sampleI)
        x_uj = self.score(sampleU, sampleJ)
        return -tf.reduce_mean(tf.math.log(tf.math.sigmoid(x_ui - x_uj)))

With a single lambda

Lambda   | Objective   | AUC                | Leaderboard

0.00001  | 0.44241133  | 0.7886660337447313 | N/A

0.0001   | 0.54610515  | 0.7520641582971066 | 0.6787

0.001    | 0.6417916   | 0.7452726561068631 | 0.5544

With two lambdas

0.00001, 0.0001 | 0.7560826296240774 | 0.6977

0.0001, 0.00001 | 0.7919284964772884 | 0.7207*

0.00001, 0.000001 | 0.7764748746648007 | 0.7095

lr=0.1, K=5, lambda=0.00001, threshold=0.59, leader=0.7285

K=6 AUC: 0.7933193424274209

In [662]:
modelBPR = BPRbatch(6, 0.00001) # need to experiment lambda too

def trainingStepBPR(model, interactions):
    Nsamples = 50000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleJ = [],[],[]
        for _ in range(Nsamples):
            u,i,_ = random.choice(interactions) # positive sample
            j = random.choice(items) # negative sample
            while j in itemsPerUser[u]:
                j = random.choice(items)
            sampleU.append(userIDs[u])
            sampleI.append(itemIDs[i])
            sampleJ.append(itemIDs[j])
        
        loss = model(sampleU, sampleI, sampleJ)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for 
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()

200 epochs does not improve much compared with 100 epochs. But we got almost to the lowest objective value at 120 epochs. To avoid underfitting though, we still use 200 epochs when training.

In [663]:
# Run 200 batches of gradient descent, the code below has been run for several times
for i in range(200):
    obj = trainingStepBPR(modelBPR, interactionsTrain)
    if (i % 20 == 19): print("Iteration " + str(i+1) + ", objective = " + str(obj))

Iteration 20, objective = 0.4877563
Iteration 40, objective = 0.45658654
Iteration 60, objective = 0.4510499
Iteration 80, objective = 0.44498116
Iteration 100, objective = 0.4404919
Iteration 120, objective = 0.43773398
Iteration 140, objective = 0.43912643
Iteration 160, objective = 0.43745488
Iteration 180, objective = 0.43903378
Iteration 200, objective = 0.43722948


In [469]:
len(userSet), len(gameSet), len(playedSet)

(6710, 2437, 174999)

In [471]:
interactionsTestPerUser = defaultdict(set)
itemSet = set()
for u,i,_ in interactionsTest:
    interactionsTestPerUser[u].add(i)
    itemSet.add(i)

In [472]:
def AUCu(u, N): # N samples per user
    win = 0
    if N > len(interactionsTestPerUser[u]):
        N = len(interactionsTestPerUser[u])
    positive = random.sample(interactionsTestPerUser[u],N)
    negative = random.sample(gameSet.difference(interactionsTestPerUser[u]),N)
    for i,j in zip(positive,negative):
        si = modelBPR.predict(userIDs[u], itemIDs[i]).numpy()
        sj = modelBPR.predict(userIDs[u], itemIDs[j]).numpy()
        if si > sj:
            win += 1
    return win/N

In [473]:
def AUC():
    av = []
    for u in interactionsTestPerUser:
        av.append(AUCu(u, 10))
    return sum(av) / len(av)

In [664]:
AUC()

/var/folders/8f/_6h5xprs32dfmm3jfs0g7k880000gn/T/ipykernel_38285/3135776419.py:5: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  positive = random.sample(interactionsTestPerUser[u],N)
/var/folders/8f/_6h5xprs32dfmm3jfs0g7k880000gn/T/ipykernel_38285/3135776419.py:6: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  negative = random.sample(gameSet.difference(interactionsTestPerUser[u]),N)


0.7933193424274209

In [665]:
scores = []
for (label, sample) in [(1, playedValid), (0, notPlayed)]:
    for (u,g) in sample:
        score = modelBPR.predict(userIDs[u], itemIDs[g]).numpy()
        scores.append(score)

In [666]:
scores.sort()
scores.reverse()

In [667]:
scores[9999], scores[10000], scores[9999] - scores[10001]

(0.56898946, 0.56875527, 0.00036346912)

In [589]:
0.5561886 - 0.00038319826

0.55580540174

In [576]:
0.556

0.556

In [ ]:
correct = 0
p0, p1 = 0,0
scores = []
for (label, sample) in [(1, playedValid), (0, notPlayed)]:
    for (u,g) in sample:
        score = modelBPR.predict(userIDs[u], itemIDs[g]).numpy()
        scores.append(score)
        
        maxSim = 0
        users = set(hoursPerItem[g])
        for g2,_ in hoursPerItem[u]:
            sim = CosineSet(users, set(hoursPerItem[g2]))
            if sim > maxSim:
                maxSim = sim
        pred = 0
        if maxSim > 0.0025 or len(hoursPerItem[g]) > 60:
            pred = 1
            p1 += 1
        else:
            p0 += 1
        if pred == label:
            correct += 1
acc = correct / (len(playedValid) + len(notPlayed))

In [226]:
scores = [modelBPR.predict(userIDs[u], itemIDs[i]).numpy() for u,i,l in interactionsTrain]

In [241]:
scores.sort()
scores[1000]

-0.8930664

In [239]:
len(scores), max(scores), min(scores)

(165000, 1.2336296, -1.0530381)

In [229]:
globalAverage = sum(scores) / len(scores)
globalAverage

0.33681310097428785

In [441]:
# Generate negative labels
notPlayed = set()
for u,g,d in hoursValid:
    g = random.choice(lGameSet)
    while (u,g) in playedSet or (u,g) in notPlayed:
        g = random.choice(lGameSet)
    notPlayed.add((u,g, 1))

playedValid = set()
for u,g,r in hoursValid:
    playedValid.add((u,g))

In [264]:
len(interactionsTest)

9999

In [242]:
itemScorePerUser = defaultdict(list)

In [243]:
for u,i,l in interactionsTrain:
    score = modelBPR.predict(userIDs[u], itemIDs[i]).numpy()
    itemScorePerUser[u].append(score)

In [249]:
for u in itemScorePerUser:
    itemScorePerUser[u].sort()
    itemScorePerUser[u].reverse()

In [245]:
lengths = [len(value) for key,value in itemsPerUser.items()]

In [247]:
sum(lengths) / len(lengths)

24.59016393442623

In [442]:
gameCount = defaultdict(int)
totalPlayed = 0

for user,game,_ in readJSON("train.json.gz"):
    gameCount[game] += 1
    totalPlayed += 1

mostPopular = [(gameCount[x], x) for x in gameCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalPlayed/1.4844999999999906: break

In [444]:
notPlayed = set()
for u,g,d in hoursValid:
    g = random.choice(lGameSet)
    while (u,g) in playedSet or (u,g) in notPlayed:
        g = random.choice(lGameSet)
    notPlayed.add((u,g))

playedValid = set()
for u,g,r in hoursValid:
    playedValid.add((u,g))

In [445]:
def pred_eval(threshold=0):
    correct = 0
    for label,sample in [(1, playedValid), (0, notPlayed)]:
        for (u,b) in sample:
            pred = 0
            if b in return1:
                pred = 1
            if u in userIDs and g in itemIDs:
                if modelBPR.predict(userIDs[u], itemIDs[g]).numpy() > threshold:
                    pred = 1
            if label == pred: correct += 1
    return correct / (len(playedValid) + len(notPlayed))

In [509]:
# bestAcc = 0
# bestThreshold = None
# for threshold in np.arange(-1, 1, 0.1):
#     acc = pred_eval(threshold)
#     if acc > bestAcc:
#         bestAcc = acc
#         bestThreshold = threshold

In [417]:
bestThreshold, bestAcc

(0.8999999999999995, 0.5343534353435343)

In [388]:
correct / (len(playedValid) + len(notPlayed))

0.5006000600060005

In [383]:
(len(playedValid) + len(notPlayed))

19998

In [447]:
notPlayed = set()
for u,g,d in hoursValid:
    g = random.choice(lGameSet)
    while (u,g) in playedSet or (u,g) in notPlayed:
        g = random.choice(lGameSet)
    notPlayed.add((u,g))

playedValid = set()
for u,g,r in hoursValid:
    playedValid.add((u,g))

In [ ]:
correct / (len(playedValid) + len(notPlayed))

0.7031203120312031

k=5 --> thre=0.556

k=3 --> thre=0.53543

k=2 --> thre=0.49670 | 0.722

k=4 --> thre=0.5456 | 0.7278

k=6 --> 0.7286

In [668]:
predictions = open("predictions_Played.csv", 'w')
for l in open("pairs_Played.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,g = l.strip().split(',')
    pred = 0
    if (u,g,1) in interactionsTrain:
        pred = 1
    if u in userIDs and g in itemIDs:
        if modelBPR.predict(userIDs[u], itemIDs[g]).numpy() > 0.5689:
            pred = 1
    else:
        if g in return1:
            pred = 1
    _ = predictions.write(u + ',' + g + ',' + str(pred) + '\n')

predictions.close()

## Fine Tuning

BPR
* Learning rate for Adam. --> Done. 0.1 is the best one.
* Try out SGD and fine-tune the parameter --> Sucks, gone
* Number of Epoch to train the model. --> 200.
* Fine-tune the lambda -- maybe split into two lambdas. --> Done. Stick with the original one.
* Threshold for label prediction.
* Fine-tune the K-value of the BPR.
* Ensemble different model predict (incorporate jaccard maybe?).

LFM

* Optimizer choice (Adam or SGD).
* Learning rate for the optimizer.
* Introduce two gamma terms.
* K value (less than 5).
* Alpha (offset term).
* Regularizer (introduce two regularizers instead of one).

## Part 2: Hours prediction